In [2]:
!pip install -U git+https://github.com/andys0975/google-scholar-crawler.git

  Cloning https://github.com/andys0975/google-scholar-crawler.git to /tmp/pip-req-build-ahx265f2
  Running command git clone -q https://github.com/andys0975/google-scholar-crawler.git /tmp/pip-req-build-ahx265f2
  Stored in directory: /tmp/pip-ephem-wheel-cache-nr_3phc7/wheels/50/54/9b/0c76ff1031dd62cf7bae28fca9d70f36b4e3fef4eba9580871
Successfully built scholarpy


In [0]:
import os
from threading import Timer

import pandas as pd

from Crawler import Crawler

In [0]:
keywords = [
    'alzheimer',
    'randomized',
    'trial'
]

In [5]:
crawler = Crawler(
    key = keywords,
    exclude=['amyloid'],
    page_list=1, # range(5), [1,4,6,9,10]
    author = None, # 'Po-Hsiang Tsui'
    source = None, # 'Google Patents', 'arxiv', 'radiology'
    only_in_title=False,
    patent = True,
    sort_by_date = True,
    y_since = 2005, # doi was introduced in 2000
    y_until = None,
    count_per_page = 10,
    key_score={'p': 1, 'n': -3, 'none': -5}, # p for key; n for exclude; none for no title/abstract
    weighting={'title': 1.5, 'content': 1},
    score_threshold=0
)

Base url: http://scholar.google.com.tw/scholar?q=alzheimer+randomized+trial&as_sdt=0,5&scisbd=1&as_ylo=2005&num=10


In [0]:
%%time
results = crawler.crawl()

### Turn to page 1 ###
1 https://doi.org/10.1161/JAHA.119.011938
2 2 DOI not in PubMed at line 217
2 https://doi.org/10.3233/JAD-180525
3 list index out of range at line 256
3 list index out of range at line 263
3 'abstract' at line 266
3 abstract may not complete
4 abstract may not complete
5 abstract may not complete
7 https://doi.org/10.1186/s13195-019-0492-1
8 list index out of range at line 221
8 https://doi.org/10.1186/s13195-019-0484-1
9 list index out of range at line 222
9 https://doi.org/10.1212/WNL.0000000000007418
10 list index out of range at line 256
10 list index out of range at line 263
10 'abstract' at line 266
10 abstract may not complete
Total acquired number: 10
CPU times: user 2.21 s, sys: 54.5 ms, total: 2.26 s
Wall time: 23.2 s


In [0]:
results[0]

{'abstract': 'background hypertension is common among patients with alzheimer disease. because this group has been excluded from hypertension trials, evidence regarding safety of treatment is lacking. this secondary analysis of a randomized controlled trial assessed whether antihypertensive treatment increases the prevalence of orthostatic hypotension (oh) in patients with alzheimer disease. methods and results four hundred seventy-seven patients with mild-to-moderate alzheimer disease were randomized to the calcium-channel blocker nilvadipine 8\xa0mg/day or placebo for 78\xa0weeks. presence of oh (blood pressure drop ≥20/≥10\xa0mm\xa0hg after 1 minute of standing) and oh-related adverse events (dizziness, syncope, falls, and fractures) was determined at 7 follow-up visits. mean age of the study population was 72.2±8.2\xa0years and mean mini-mental state examination score was 20.4±3.8. baseline blood pressure was 137.8±14.0/77.0±8.6\xa0mm\xa0hg. grade i hypertension was present in 53.4

In [0]:
df = pd.DataFrame(columns=['title', 'type', 'authors', 'year', 'journal', 'abstract', 'url', 'pdf_link', 'cite', 'score', 'require', 'complete'])

In [0]:
for result in results:
    df = df.append(result, ignore_index=True)

In [0]:
df

,title,type,authors,year,journal,abstract,url,pdf_link,cite,score,require,complete
0,Blood Pressure Lowering With Nilvadipine in Pa...,paper,"de Heus Rianne A A, Donders Rogier, Santoso An...",2019,Journal of the American Heart Association,background hypertension is common among patien...,https://doi.org/10.1161/JAHA.119.011938,https://www.ahajournals.org/doi/pdf/10.1161/JA...,0,4.5,True,True
1,Neuropsychological Decline Improves Prediction...,paper,"Nation Daniel A., Ho Jean K., Dutt Shubir, Han...",None,Journal of Alzheimer's Disease,Background: A clinical diagnosis of cognitive ...,https://doi.org/10.3233/JAD-180525,NaN,0,1.5,True,True
2,Visualization of natural landscapes and select...,paper,,2019,Taylor & Francis,6 天前 - … Abstract. Abstract. Alzheimer's Dise...,https://www.tandfonline.com/doi/full/10.1080/2...,NaN,0,3.0,True,False
3,Feasibility and effects of galantamine on cogn...,paper,NaN,2019,Elsevier,8 天前 - … is an acetylcholinesterase inhibitor...,https://www.sciencedirect.com/science/article/...,NaN,0,3.0,True,False
4,A randomized trial of naproxen to slow progres...,paper,NaN,2019,AAN Enterprises,18 天前 - This study tested the efficacy of low-...,https://n.neurology.org/content/neurology/92/1...,NaN,0,5.5,True,False
5,Electronic Health Records Based Prediction of ...,pre-print,"Ji Hwan Park, Han Eol Cho, Jong Hun Kim, Melan...",2019,biorxiv.org,background: accurate prediction of future inci...,https://www.biorxiv.org/content/10.1101/625582...,https://www.biorxiv.org/content/biorxiv/early/...,0,3.5,True,True
6,Safety and tolerability of donepezil 23 mg wit...,paper,"Hong Yun Jeong, Han Hyun Jeong, Youn Young Chu...",2019,Alzheimer's research & therapy,high-dose donepezil is currently prescribed fo...,https://doi.org/10.1186/s13195-019-0492-1,https://alzres.biomedcentral.com/articles/10.1...,0,5.5,True,True
7,Effects of spermidine supplementation on cogni...,paper,"Miranka Wirth, Claudia Schwarz, Gloria Benson,...",2019,Alzheimer's Research & Therapy,Given the global increase in the aging populat...,https://doi.org/10.1186/s13195-019-0484-1,https://alzres.biomedcentral.com/articles/10.1...,0,7.5,True,True
8,Can naproxen slow the progression of Alzheimer...,paper,"Steven Karceski, Steven Karceski",2019,Neurology,In their study “INTREPAD: A randomized trial o...,https://doi.org/10.1212/WNL.0000000000007418,NaN,0,4.5,True,True
9,Estudio comparativo sobre la glicación de la l...,paper,,2019,dialnet.unirioja.es,"22 天前 - … Tal es así, que la tendencia a desa...",https://dialnet.unirioja.es/servlet/articulo?c...,https://dialnet.unirioja.es/servlet/articulo?c...,0,3.0,True,False


In [0]:
class RepeatedTimer(object):
    def __init__(self, interval, crawler, results, id_list):
        self._timer = None
        self.counter = 0    ###
        self.interval = interval
        self.crawler = crawler
        self.results = results
        self.id_list = id_list
        self.is_running = False
        self.start()
    
    def schedule(self):
        try:
            for result in self.crawler.crawl():
                if result['url'] in self.id_list:
                    continue
                else:
                    self.id_list += [result['url']]    ###
                    self.results += [result]
            self.counter += 1
        except: True

    def _run(self):
        self.is_running = False
        self.start()
        self.schedule()

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False
        return self.counter, self.results, self.id_list

In [0]:
results = []; id_list = []

In [0]:
rt = RepeatedTimer(120, crawler, results, id_list)

In [0]:
counter, results, id_list = rt.stop()

In [0]:
counter

6

For Colab only

In [10]:
!apt-get install -y -qq software-properties-common module-init-tools
!add-apt-repository ppa:alessandro-strada/ppa
!apt-get update
!apt-get install google-drive-ocamlfuse
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

 Mount Google Drive on Ubuntu (via FUSE)
 More info: https://launchpad.net/~alessandro-strada/+archive/ubuntu/ppa
Press [ENTER] to continue or Ctrl-c to cancel adding it.

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:7 http://ppa.launchpad.net/alessandro-strada/ppa/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/gr

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [12]:
!ls drive

 04_Basics.pdf
 27174_Ai-99-Ai-Lab-a-wy-cj-1.odt
 32330_MetsOrigin.xlsx
 34243_mail.odt
 Arduino.ods
'Brain tumours.odt'
 CBS.odt
'Colab Notebooks'
 draw.io
'Functional MRI.pdf'
'Kami Uploads'
 LIDC-IDRI_MetaData.ods
'lidc-idri nodule counts (6-23-2015).ods'
 MetsOrigin.xlsx
 paperlab
'PETMR 簡介_201808.pdf'
 ProstateDiagnosis_metadata-05-07-2012.ods
 Screencastify
'Seminar(III) 評分與評論.docx'
 tcia-diagnosis-data-2012-04-20.ods
'thalamic relay.odt'
 VAFL20180326.ods.ods
 撰稿.odt
 無標題文件.odt
'無標題文件.odt (2f1d7019)'
 班表副本.ods
 鄭仕群AI合作.pdf


In [0]:
os.chdir("drive/paperlab/") 

In [0]:
df.to_excel('paperlab.xlsx')

In [0]:
df = pd.read_excel("paperlab.xlsx")

In [15]:
df

,Unnamed: 0,title,type,authors,year,journal,abstract,url,pdf_link,cite,score,require,complete
0,0,Blood Pressure Lowering With Nilvadipine in Pa...,paper,"de Heus Rianne A A, Donders Rogier, Santoso An...",2019.0,Journal of the American Heart Association,background hypertension is common among patien...,https://doi.org/10.1161/JAHA.119.011938,https://www.ahajournals.org/doi/pdf/10.1161/JA...,0,4.5,True,True
1,1,Neuropsychological Decline Improves Prediction...,paper,"Nation Daniel A., Ho Jean K., Dutt Shubir, Han...",NaN,Journal of Alzheimer's Disease,Background: A clinical diagnosis of cognitive ...,https://doi.org/10.3233/JAD-180525,NaN,0,1.5,True,True
2,2,Visualization of natural landscapes and select...,paper,NaN,2019.0,Taylor & Francis,6 天前 - … Abstract. Abstract. Alzheimer's Dise...,https://www.tandfonline.com/doi/full/10.1080/2...,NaN,0,3.0,True,False
3,3,Feasibility and effects of galantamine on cogn...,paper,NaN,2019.0,Elsevier,8 天前 - … is an acetylcholinesterase inhibitor...,https://www.sciencedirect.com/science/article/...,NaN,0,3.0,True,False
4,4,A randomized trial of naproxen to slow progres...,paper,NaN,2019.0,AAN Enterprises,18 天前 - This study tested the efficacy of low-...,https://n.neurology.org/content/neurology/92/1...,NaN,0,5.5,True,False
5,5,Electronic Health Records Based Prediction of ...,pre-print,"Ji Hwan Park, Han Eol Cho, Jong Hun Kim, Melan...",2019.0,biorxiv.org,background: accurate prediction of future inci...,https://www.biorxiv.org/content/10.1101/625582...,https://www.biorxiv.org/content/biorxiv/early/...,0,3.5,True,True
6,6,Safety and tolerability of donepezil 23 mg wit...,paper,"Hong Yun Jeong, Han Hyun Jeong, Youn Young Chu...",2019.0,Alzheimer's research & therapy,high-dose donepezil is currently prescribed fo...,https://doi.org/10.1186/s13195-019-0492-1,https://alzres.biomedcentral.com/articles/10.1...,0,5.5,True,True
7,7,Effects of spermidine supplementation on cogni...,paper,"Miranka Wirth, Claudia Schwarz, Gloria Benson,...",2019.0,Alzheimer's Research & Therapy,Given the global increase in the aging populat...,https://doi.org/10.1186/s13195-019-0484-1,https://alzres.biomedcentral.com/articles/10.1...,0,7.5,True,True
8,8,Can naproxen slow the progression of Alzheimer...,paper,"Steven Karceski, Steven Karceski",2019.0,Neurology,In their study “INTREPAD: A randomized trial o...,https://doi.org/10.1212/WNL.0000000000007418,NaN,0,4.5,True,True
9,9,Estudio comparativo sobre la glicación de la l...,paper,NaN,2019.0,dialnet.unirioja.es,"22 天前 - … Tal es así, que la tendencia a desa...",https://dialnet.unirioja.es/servlet/articulo?c...,https://dialnet.unirioja.es/servlet/articulo?c...,0,3.0,True,False


In [0]:
id_list = df.url.tolist()

In [19]:
id_list

['https://doi.org/10.1161/JAHA.119.011938',
 'https://doi.org/10.3233/JAD-180525',
 'https://www.tandfonline.com/doi/full/10.1080/21711976.2019.1602339',
 'https://www.sciencedirect.com/science/article/pii/S009130571930098X',
 'https://n.neurology.org/content/neurology/92/18/e2070.abridgement.pdf',
 'https://www.biorxiv.org/content/10.1101/625582v1.abstract',
 'https://doi.org/10.1186/s13195-019-0492-1',
 'https://doi.org/10.1186/s13195-019-0484-1',
 'https://doi.org/10.1212/WNL.0000000000007418',
 'https://dialnet.unirioja.es/servlet/articulo?codigo=6892344']